In [1]:
import torch

from transformers import BertTokenizer
from PIL import Image
import argparse

from models import caption
from datasets import coco, utils
from configuration import Config
import os 

In [3]:
model = torch.hub.load('saahiluppal/catr', 'v3', pretrained=True)

Using cache found in C:\Users\86158/.cache\torch\hub\saahiluppal_catr_master
D:\3D66\anaconda\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
D:\3D66\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
start_token = tokenizer.convert_tokens_to_ids(tokenizer._cls_token)
end_token = tokenizer.convert_tokens_to_ids(tokenizer._sep_token)

In [16]:
def create_caption_and_mask(start_token, max_length):
    caption_template = torch.zeros((1, max_length), dtype=torch.long)
    mask_template = torch.ones((1, max_length), dtype=torch.bool)

    caption_template[:, 0] = start_token
    mask_template[:, 0] = False

    return caption_template, mask_template


@torch.no_grad()
def evaluate(image_path):
    image = Image.open(image_path)
    image = coco.val_transform(image)
    image = image.unsqueeze(0)
    #print(image)
    model.eval()
    for i in range(config.max_position_embeddings - 1):
        predictions = model(image, caption, cap_mask)
        predictions = predictions[:, i, :]
        predicted_id = torch.argmax(predictions, axis=-1)

        if predicted_id[0] == 102:
            return caption

        caption[:, i+1] = predicted_id[0]
        cap_mask[:, i+1] = False

    return caption

In [15]:
config = Config()
caption, cap_mask = create_caption_and_mask(start_token, config.max_position_embeddings)

In [7]:
import string
image_dataset_path = 'E:/jupterfiles/CV/image captioning/IMsimple/data/Flickr8k_Dataset/Flicker8k_Dataset'
caption_dataset_path = 'E:/jupterfiles/CV/image captioning/IMsimple/data/Flickr8k_text/Flickr8k.token.txt'

In [8]:
captions_dict1 = {}
    
# iterate through the file
for caption in open(caption_dataset_path):
        
    # Splitting the name of image file and image caption
    tokens = caption.split()
    caption_id, caption_text = tokens[0].split('.')[0], tokens[1:]
    caption_text = ' '.join(caption_text) #concat to be a string
    if caption_id not in captions_dict1:
        captions_dict1[caption_id] = []
    captions_dict1[caption_id].append(caption_text)

In [9]:
captions_dict1

{'1000268201_693b08cb0e': ['A child in a pink dress is climbing up a set of stairs in an entry way .',
  'A girl going into a wooden building .',
  'A little girl climbing into a wooden playhouse .',
  'A little girl climbing the stairs to her playhouse .',
  'A little girl in a pink dress going into a wooden cabin .'],
 '1001773457_577c3a7d70': ['A black dog and a spotted dog are fighting',
  'A black dog and a tri-colored dog playing with each other on the road .',
  'A black dog and a white dog with brown spots are staring at each other in the street .',
  'Two dogs of different breeds looking at each other on the road .',
  'Two dogs on pavement moving toward each other .'],
 '1002674143_1b742ab4b8': ['A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .',
  'A little girl is sitting in front of a large painted rainbow .',
  'A small girl in the grass plays with fingerpaints in front of a white canvas with a rainbow on it .',
  'There is a gi

In [10]:
# load the caption file & read it
def load_caption_file(path):
    
    # dictionary to store captions
    captions_dict = {}
    
    # iterate through the file
    for caption in open(path):
        
        # Splitting the name of image file and image caption
        tokens = caption.split()
        caption_id, caption_text = tokens[0].split('.')[0], tokens[1:]
        caption_text = ' '.join(caption_text) #concat to be a string
        
        # save it in the captions dictionary
        if caption_id not in captions_dict:
            captions_dict[caption_id] = caption_text
        
    return captions_dict

# call the function
captions_dict = load_caption_file(caption_dataset_path)

In [11]:
# dictionary to store the cleaned captions
new_captions_dict = {}

# prepare translation table for removing punctuation.
table = str.maketrans('', '', string.punctuation)

# loop through the dictionary
for caption_id, caption_text in captions_dict.items():
    # tokenize the caption_text
    caption_text = caption_text.split()
    # convert it into lower case
    caption_text = [token.lower() for token in caption_text]
    # remove punctuation from each token
    caption_text = [token.translate(table) for token in caption_text]
    # remove all the single letter tokens like 'a', 's'
    caption_text = [token for token in caption_text if len(token)>1]
    # store the cleaned captions
    new_captions_dict[caption_id] = 'startseq ' + ' '.join(caption_text) + ' endseq'

In [12]:
# delete unwanted captions which do not match any images
del captions_dict

In [13]:
caption_images_list = []

image_index = list(new_captions_dict.keys())

caption_images_list = [ image.split('.')[0] for image in os.listdir(image_dataset_path) if image.split('.')[0] in image_index ]
# Total images along with captionslen
print(len(caption_images_list))

8091


In [14]:
 import transformers
transformers.__version__

'2.11.0'

In [17]:
#output = evaluate(os.path.join(image_dataset_path,str(image_index[3])+'.jpg'))
output = evaluate(os.path.join(image_dataset_path,'3613667665_1881c689ea.jpg'))
result = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
result.capitalize()

E:\jupterfiles\CV\image captioning\catr\models\position_encoding.py:38: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])
1
torch.Size([1, 2048, 13, 19])


'A dog running on the beach with a ball in its mouth.'

In [36]:
from nltk.translate.bleu_score import sentence_bleu,corpus_bleu
from tqdm import tqdm
from nltk.translate.bleu_score import SmoothingFunction
smooth = SmoothingFunction() # 定义平滑函数对象
test_list = image_index[0:100]
references = []
hypothese = []
for image in tqdm(test_list):
    captions = captions_dict1[image]
#     captions.replace('startseq','')
#     captions.replace('endseq','')
    references.append(captions)
    output = evaluate(os.path.join(image_dataset_path,str(image)+'.jpg'))
    result = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
    hypothese.append(result.capitalize())


100%|██████████| 100/100 [10:03<00:00,  6.03s/it]


In [37]:
bleu1 = corpus_bleu(references, hypothese, weights=(1, 0, 0, 0), smoothing_function=smooth.method7)
bleu2 = corpus_bleu(references, hypothese, weights=(0.5, 0.5, 0, 0), smoothing_function=smooth.method7)
bleu3 = corpus_bleu(references, hypothese, weights=(0.3, 0.3, 0.3, 0), smoothing_function=smooth.method7)
bleu4 = corpus_bleu(references,hypothese,weights=(0.25,0.25,0.25,0.25),smoothing_function=smooth.method7)

In [38]:
#100
print(str(bleu1)+" "+str(bleu2)+" "+str(bleu3)+" "+str(bleu4))

1.13128917806643 0.9210810218339969 0.7873586358234242 0.6452057076060979


In [27]:
#10
print(str(bleu1)+" "+str(bleu2)+" "+str(bleu3)+" "+str(bleu4))

0.7156521662817381 0.6222952517739581 0.5655932142773119 0.4241232777652718


In [18]:
a = 'A man lays on a bench while his dog sits by him .'
b = 'A man lays on the bench to which a white dog is also tied .'
c = 'a man sleeping on a bench outside with a white and black dog sitting next to him'
d = 'A shirtless man lies on a park bench with his dog .'
e = 'man laying on bench holding leash of dog sitting on ground'
f = 'A man laying on a bench with a dog.'
re = [[a,b,c,d,e]] 
hy = [f]
bleu1 = corpus_bleu(re,hy, weights=(1, 0, 0, 0), smoothing_function=smooth.method7)
bleu2 = corpus_bleu(re,hy, weights=(0.5, 0.5, 0, 0), smoothing_function=smooth.method7)
bleu3 = corpus_bleu(re,hy, weights=(0.3, 0.3, 0.3, 0), smoothing_function=smooth.method7)
blue4 = corpus_bleu(re,hy, weights=(0.25,0.25,0.25,0.25),smoothing_function=smooth.method7)

In [19]:
print(str(bleu1)+" "+str(bleu2)+" "+str(bleu3)+" "+str(bleu4))

0.8871882962236403 0.8006184406266234 0.7399492742811946 0.4497164161018573


In [17]:
hy = ['boy is jumping and the grass']
bleu1 = corpus_bleu(re,hy, weights=(1, 0, 0, 0), smoothing_function=smooth.method7)
bleu2 = corpus_bleu(re,hy, weights=(0.5, 0.5, 0, 0), smoothing_function=smooth.method7)
bleu3 = corpus_bleu(re,hy, weights=(0.3, 0.3, 0.3, 0), smoothing_function=smooth.method7)
blue4 = corpus_bleu(re,hy,weights=(0.25,0.25,0.25,0.25),smoothing_function=smooth.method7)
print(str(bleu1)+" "+str(bleu2)+" "+str(bleu3)+" "+str(bleu4))

0.5777534467828813 0.4697214601416387 0.38617040870059266 0.4497164161018573


In [35]:
a = torch.rand(1, 2048, 13, 19)
print(a.shape)
b = a.flatten(2)
print(b.shape)
c = a.flatten(2).permute(2,0,1)
print(c.shape)

torch.Size([1, 2048, 13, 19])
torch.Size([1, 2048, 247])
torch.Size([247, 1, 2048])
